# Using CLDK to explain Java methods

In this tutorial, we will use CLDK to explain or generate code summary for a Java method. You'll explore some of the benefits of using CLDK to perform quick and easy program analysis and build an LLM-based code summarizer. By the end of this tutorial, you will have implemented such a tool and generated code summary for a Java method.

Specifically, you will learn how to perform the following tasks on a Java application to create LLM prompts for code summarization:

1. Create a new instance of the CLDK class.
2. Create an analysis object for the target Java application.
3. Iterate over all files in the application.
4. Iterate over all classes in a file.
5. Initialize treesitter utils for the class content.
6. Iterate over all methods in a class.
7. Get the code body of a method.
8. Sanitize the class for prompting the LLM.

We will write a couple of helper methods to (1) format the LLM instruction for summarizing a given target method and (2) prompt the LLM via Ollama. We will then use CLDK to go through an application and generate the summary for the target method.

## Prequisites

Before we get started, let's make sure you have the following installed:

1. Python 3.11 or later (you can use [pyenv](https://github.com/pyenv/pyenv) to install Python)
2. Java 11 or later (you can use [SDKMAN!](https://sdkman.io) to instal Java)
3. Ollama 0.3.4 or later (you can get Ollama here: [Ollama download](https://ollama.com/download))

We will use Ollama to spin up a local [Granite code model](https://ollama.com/library/granite-code), which will serve as our LLM for this turorial.

### Download Granite code model

After starting the Ollama server, please download the latest version of the Granite code 8b-instruct model by running the following command. There are other Granite code models available, but for this tutorial, we will use Granite code 8b-instruct. If you prefer to use a different Granite code model, you can replace `8b-instruct` with the tag of another version (see [Granite code tags](https://ollama.com/library/granite-code/tags)).

In [ ]:
%%bash
ollama pull granite-code:8b-instruct

 Let's make sure the model is downloaded by running the following command:

In [ ]:
%%bash
ollama run granite-code:8b-instruct \"Write a python function to print 'Hello, World!'

### Install Ollama Python SDK

In [ ]:
pip install ollama

### Install CLDK
CLDK is avaliable at https://github.com/IBM/codellm-devkit. You can install it by running the following command:

In [ ]:
pip install git+https://github.com/IBM/codellm-devkit.git

### Step 1: Get the sample Java application
For this tutorial, we will use [Apache Commons CLI](https://github.com/apache/commons-cli) as the sample Java application. You can download the source code to a temporary directory by running the following command:

In [ ]:
%%bash
wget https://github.com/apache/commons-cli/archive/refs/tags/rel/commons-cli-1.7.0.zip -O /tmp/commons-cli-1.7.0.zip && unzip -o /tmp/commons-cli-1.7.0.zip -d /tmp

The project will now be extracted to `/tmp/commons-cli-rel-commons-cli-1.7.0`.
<!-- We'll remove these files later, so don't worry about the location. -->

## Generate code summary

Code summarization or code explanation is the task of converting code written in a programming language to natural language. It has several benefits, such as understanding code without looking at its intrinsic details, documenting code for better maintenance, etc. To perform code summarization, one needs to understand the basic details of code implementation, and use that knowledge to generate the summary using various AI-based approaches. In this tutorial, we will use LLMs, specifically Granite code 8b-instruct. We will show how a developer can easily use CLDK to analyze code by calling various APIs without having to implement such analyses.

Step 1: Add the neccessary imports

In [ ]:
import ollama
from cldk import CLDK
from cldk.analysis import AnalysisLevel

Step 2: Define a function for creating the LLM prompt, which instructs the LLM to summarize a Java method and includes relevant code for the task.

In [ ]:
def format_inst(code, focal_method, focal_class, language):
    """
    Format the instruction for the given focal method and class.
    """
    inst = f"Question: Can you write a brief summary for the method `{focal_method}` in the class `{focal_class}` below?\n"

    inst += "\n"
    inst += f"```{language}\n"
    inst += code
    inst += "```" if code.endswith("\n") else "\n```"
    inst += "\n"
    return inst

Step 3: Define a function to call the LLM (in this case, Granite code 8b-instruct) using Ollama.

In [ ]:
def prompt_ollama(message: str, model_id: str = "granite-code:8b-instruct") -> str:
    """Prompt local model on Ollama"""
    response_object = ollama.generate(model=model_id, prompt=message, options={"temperature":0.2})
    return response_object["response"]

Step 4: Create an instance of CLDK and provide the programming language of the source code.

In [ ]:
# Create an instance of CLDK for Java analysis
cldk = CLDK(language="java")

Step 5: Select the analysis engine and analysis level. CLDK uses different analysis engines---[CodeAnalyzer](https://github.com/IBM/codenet-minerva-code-analyzer) (built over [WALA](https://github.com/wala/WALA) and [JavaParser](https://github.com/javaparser/javaparser)), [Treesitter](https://tree-sitter.github.io/tree-sitter/), and [CodeQL](https://codeql.github.com/) (future)---with CodeAnalyzer being the default analysis engine. CLDK supports different analysis levels: (1) symbol table, (2) call graph, (3) program dependency graph, and (4) system dependency graph. The analysis level can be selected using the `AnalysisLevel` enumerated type. For this example, we select the symbol-table analysis level, with CodeAnalyzer as the default analysis engine.

In [ ]:
# Create an analysis object over the java application
analysis = cldk.analysis(project_path="/tmp/commons-cli-rel-commons-cli-1.7.0", analysis_level=AnalysisLevel.symbol_table)

Step 6: Iterate over all the class files and create the prompt. In this case, we want to provide a sanitized Java class in the prompt, containing only the relevant information for summarizing the target method. To illustrate, consider the floowing class:

```java
package com.ibm.org;
import A.B.C.D;
...
public class Foo {
 // code comment
 public void bar(){ 
    int a;
    a = baz();
    // do something
    }
 private int baz()
 {
    // do something
 }
 public String dummy (String a)
 {
    // do somthing
 }   
```
Let's say we want to generate a summary for method `bar`. To understand what it does, we add the callees of this method in the prompt, which in this case includes `baz`. We remove the other methods, imports, comments, etc. All of this can be achieved with a single call to CLDK's `sanitize_focal_class` API. In this process, we also use Treesitter to analyze the code.  After creating the sanitized code, we call the previously defined `format_inst` method to create the LLM prompt and pass the prompt to `prompt_ollama` to generate the method summary.

In [ ]:
# For simplicity, we run the code summarization for a single class and method (this filter can be removed to run this code over the entire application)
target_class = "org.apache.commons.cli.GnuParser"
target_method = "flatten(Options, String[], boolean)"

# Iterate over all classes in the application
for class_name in analysis.get_classes():
    if class_name == target_class:
        class_file_path = analysis.get_java_file(qualified_class_name=class_name)

        # Read code for the class
        with open(class_file_path, 'r') as f:
            code_body = f.read()

        # Initialize treesitter utils for the class file content
        tree_sitter_utils = cldk.tree_sitter_utils(source_code=code_body)
                
        # Iterate over all methods in class
        for method in analysis.get_methods_in_class(qualified_class_name=class_name):
            if method == target_method:
        
                # Get all the method details
                method_details = analysis.get_method(qualified_class_name=class_name,
                                                     qualified_method_name=method)
                
                # Sanitize the class for analysis with respect to the target method
                sanitized_class = tree_sitter_utils.sanitize_focal_class(method_details.declaration)
        
                # Format the instruction for the given target method and class
                instruction = format_inst(
                    code=sanitized_class,
                    focal_method=method_details.declaration,
                    focal_class=class_name.split('.')[-1],
                    language="java"
                )
    
                print(f"Instruction:\n{instruction}\n")
                print(f"Generating code summary . . .\n")
                
                # Prompt the local model on Ollama
                llm_output = prompt_ollama(
                    message=instruction
                )
        
                # Print the LLM output
                print(f"LLM Output:\n{llm_output}")